In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd 

import h2o
from h2o.automl import H2OAutoML

TRAIN_PATH = "../input/tabular-playground-series-may-2022/train.csv"
TEST_PATH = "../input/tabular-playground-series-may-2022/test.csv"
SAMPLE_SUBMISSION_PATH = "../input/tabular-playground-series-may-2022/sample_submission.csv"
SUBMISSION_PATH = "submission.csv "

ID = "id"
TARGET = "target"

NEW_TRAIN_PATH = "train.csv"
NEW_TEST_PATH = "test.csv"

SEED_LIST = [7,77]
MAX_RUNTIME_SECS = 60 * 3


In [ ]:
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

for df in [train, test]:
    # Extract the 10 letters from f_27 into individual features
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
        
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))
    
train.to_csv(NEW_TRAIN_PATH,index=False)
test.to_csv(NEW_TEST_PATH,index=False)


In [ ]:
h2o.init()

train = h2o.import_file(NEW_TRAIN_PATH)
test = h2o.import_file(NEW_TEST_PATH)

x = train.columns
y = TARGET

x.remove(y)
x.remove(ID) #remove id  

pred_test = []
for selSeed in SEED_LIST:
    aml_y = H2OAutoML(max_runtime_secs=MAX_RUNTIME_SECS, seed=selSeed)
    aml_y.train(x=x, y=y, training_frame=train)

    preds_y = aml_y.predict(test)
    pred_test.append(preds_y.as_data_frame().predict) 


In [ ]:
from scipy import stats
modeResult = stats.mode(pred_test, axis=0)
final_test_pred = modeResult.mode

submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[TARGET] = final_test_pred[0]
submission.to_csv(SUBMISSION_PATH, index=False)
submission.head()
